*Более сложные задачи компьютерного зрения - сегментация (segmentation) и нахождение объектов на изображении (object detection).*

*Читает Владимир Игловиков*

---

*(лол, в 2к19 челибобер сидит на Unity)*

# Повторим, шо такое клаффисикация

Берётся какая-то картинка, потом конволюционный слой, потом активация, может быть BatchNorm, потом pulling-слои... в конце концов, pulling-слой, Fully Connected Layer и предсказание:

![alt text](attach1.png)

То есть, много слоёв, много параметров...

Но хотелось бы автjматически решать и другие задачи — например, извлекать информацию о цветах или точно занимаемых положениях объекта

# 1. Segmentation

![alt text](attach2.png)

1. **Сегментация = попиксельная классификация**; про каждый пиксель можно, для упрощения, думать как про попиксельный классификатор
2. **Не требует большого объёма тренировочных данных**; но сами данные, при этом, готовить гораздо тяжелее
3. **Все сегментационные сети — это архитектуры вида FCN**
4. На выход, как правило, выдаёт картинку такого же разрешения, что и на вход

### Метрики

![alt text](attach3.png)

# Fully Convolutional Network (FCN)

![alt text](attach4.png)

\***Dense-слои** — это другое название FC- aka полносвязных слоёв)

### FCN = Efficient Sliding Window

Как можно пытаться делать предсказание? Скользящим окном (взять какой-то кроп из куска картинки и пытаться предсказывать центральный пиксель; см. прошлые лекции). Но это очень неэффективно!

Наша полносвязная сеть выше позволет делать то же, что и скользящее окно, но гораздо более эффективно:

![alt text](attach5.png)

(картинка иллюстрирует, что если мы возьмём input чуть побольше, то мы просто получим output чуть побольше — но оно сработает)

Идея в том, что в силу работы свёрточных слоёв мы можем применять FCN к input-ам **любого размера** — и это соответствует тому, что если бы мы применяли свёрточную сеть со скользящим окном (из-за того, какая математика в convolution слоях). Это позволяет нам проанализировать картнку гораздо быстрее, чем если бы мы прогоняли RCC сеть много раз по разным разрешениям.

### 1. Classification to Segmentation

На картинке ниже — FCN8, одна из первых попыток реализовать сегментационную нейросеть на основе классификационной:

![alt text](attach6.png)

То есть, мы, "оторвав" полносвязный слой, получаем heatmap, а потом делаем upsampling до исходного размера.

\***upsampling** — повышаем разрешение, обычно применяя некоторую интерполяцию (чтобы получить менее "грубое" увеличение), как правило билинейную или бикубическую)

### 2. FCN8 to SegNet

*А давайте, вместо того, чтобы делать upsampling сразу намного, будем делать его постепенно и по чуть-чуть, перемежая его с другими слоями?* 

(upsampling -> refinement (conv + BN + ReLU) -> повторяем)

![alt text](attach7.png)

Это позволило сделать границы получамого ихображения более плавными.

### 3. SegNet to UNet

*Давайте добавим Skip Connection-ы!*

![alt text](attach8.png)

\***Skip Conenction** — переносим оригинальные данные с какого-то слоя в какой-то более поздний слой без изменений, обычно в качестве части данных, к которым конкатинируются/добавляется выход со слоя/слоёв.

Примерно то же самое, что Residiual-слои в ResNet, но здесь используются не для улучшения протекаемости градиентов, а для улучшения точности границ.

![alt text](attach10.png)

### 4. U-Net to TernausNet

*Давайте, вместо тренировки сети с нуля, загружать уже предтренированные веса!*

![alt text](attach9.png)

(TernausNet - просто пример, по такому принципу работает множество разных моделей на основе архитектиуры U-Net)

### 5. U-Net + FPN

![alt text](attach12.png)

![alt text](attach11.png)

<!-- Снимаем предсказания со всех слоёв вместо последнего, то есть работаем с heatmap-ами масками разного разрешения, получаем сегментационные маски разного размера, upsampl-им их до одного размера, конкатериуем и делаем предсказание. -->

С каждого слоя (а не только последнего) decoder-а (т.е. когда мы "разжимаем" картинку) снимаем feature map-ы (разных масштабов), upscal-им их до одного (самого большого) размера, конкатенируем и делаем предсказание.

А так как последние слои тоже тренируемые, модель "учится" учитывать признаки как маленьких, так и больших объектов — таким образом FPN помогает для определения и маленьких и больших объектов на картинках.

```
I HATE ADHD I HATE ADHD I HATE ADHD I
HATE ADHD I HATE ADHD I HATE ADHD I HATE
ADHD I HATE ADHD I HATE ADHD I HATE ADHD
I HATE ADHD I HATE ADHD I HATE ADHD I
HATE ADHD I HATE ADHD I HATE ADHD I HATE
ADHD I HATE ADHD I HATE ADHD I HATE ADHD
I HATE ADHD I HATE ADHD I HATE ADHD I
HATE ADHD I HATE ADHD I HATE ADHD I HATE
ADHD I HATE ADHD I HATE ADHD I HATE ADHD
I HATE ADHD I HATE ADHD I HATE ADHD I
HATE ADHD I HATE ADHD I HATE ADHD I HATE
ADHD I HATE ADHD I HATE ADHD I HATE ADHD
```

# Segmentation Loss Function

![alt text](attach13.png)

# 2. Detection

### Detection Metric: mAP

Median Anerage Precision

> *самая замороченная метрика ©*

![alt text](attach14.png)

### Non-Maximum Supression (NMS)

![alt text](attach15.png)

# Виды детекции (Detecion)

![alt text](attach16.png)



### One-shot Detectors

— выполняют классификацию, отрывают последний слой (т.е. получают heatmap-у), затем разбивают входные данные на сетку.

![alt text](attach17.png)

Далее берут исходную картинку, делят на регионы, и предсказывают сразу 5 значений — `[precision, x, y, w, h]` (или типа того) — в каждом регионе. Так делаеют для всех предсказаний из классификатора, прошедших некоторый threshold.

Оин из наиболее популярных One-shot детекторов — **YOLO** (You Only Look Once):

![alt text](attach18.png)

**SSD** (нет, не тверодительый накопитель) — модификация YOLO с multi-scale детекцией. Это заметно улучшает точность (см. метрику mAP (Median Average Precision)), но немного понижает скорость работы (см. метрику FPS (Frames Per Second)):

![alt text](attach19.png)

*Позже эта же архитектура появится в 3-й версии YOLO, **YOLOv3**, и будет использоваться далее и далее.*



### Two-shot Detectors

— сперва "находят" **proporsals** (предполагаемые регионы объектов), затем каждый proporsal прогоняют через классификационную сеть.

![alt text](attach20.png)

![alt text](attach21.png)

![alt text](attach22.png)

Performance:

![alt text](attach23.png)

Improvements:

![alt text](attach24.png)

# Как работать с этим всем на практике?

### Для задач сегментации:

1. Можно начать с экспериментов U-Net (или TernausNet, по сути тот же U-Net, но с возможностью брать предтренированные веса) — во всяком случае, она подойдёт для большинства задач

2. Иногда, для более солжных задач, можно взять U-Net + FPN

### Для задач детекции:

1. Для one-shot детекции — конечно же, YOLO (YOLOv5 или YOLOv8/YOLOv9)!
2. Для two-shot детекции — см. ссылку на аттаче выше, там докумнетация из Facebook Research, объясняющая выбор и работу разных видов и модификаций RCNN-ок



### That's it! ~~You're well on your way to become an osu! Rhythm Champion!~~